In [1]:
import sqlite3  
import random   
import pandas as pd  
from pandas.io.sql import read_sql  
import statsmodels.api as sm  

# Create an in-memory SQLite database and cursor
with sqlite3.connect(":memory:") as con:
    # Initialize the cursor object
    c = con.cursor()
    
    # Create the sensors table
    # The table 'sensors' will store sensor readings with columns:
    # id (unique identifier), type (sensor type), value (sensor reading), timestamp (date and time of reading)
    c.execute("""
        CREATE TABLE sensors (
            id INTEGER PRIMARY KEY,   -- Unique identifier for each sensor entry
            type TEXT,                -- Type of sensor, e.g., Temperature, Pressure, etc.
            value REAL,               -- Sensor reading, stored as a real number
            timestamp TEXT            -- Timestamp for each reading in 'YYYY-MM-DDTHH:MM' format
        )
    """)
    print("Created 'sensors' table.")  # Confirmation message for table creation
    
    # Confirm the table was created
    # Query the SQLite master table to check if the 'sensors' table exists
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sensors'")
    print("Table creation confirmed:", c.fetchall())  # Output should show 'sensors' if created successfully
    
    # Insert random data into sensors table
    # Generating random data to populate the 'sensors' table
    sensor_types = ['Temperature', 'Pressure', 'Humidity']  # Possible types of sensors
    for i in range(10):  # Loop to insert 10 random records
        sensor_type = random.choice(sensor_types)  # Randomly select a sensor type
        value = round(random.uniform(10, 100), 2)  # Random float value between 10 and 100, rounded to 2 decimals
        # Generate a random timestamp in the format 'YYYY-MM-DDTHH:MM'
        timestamp = f"2024-11-{random.randint(1, 30):02d}T{random.randint(0, 23):02d}:{random.randint(0, 59):02d}"
        # Insert the random record into the sensors table
        c.execute("INSERT INTO sensors (type, value, timestamp) VALUES (?, ?, ?)", (sensor_type, value, timestamp))
    
    # Select all results from sensors table
    # Retrieve all rows from the 'sensors' table and print as a DataFrame for better readability
    sensors_data = read_sql("SELECT * FROM sensors", con)
    print("All data in 'sensors' table:")
    print(sensors_data)  # Print all the data from the table
    
    # Drop the sensors table and confirm
    # Deleting the 'sensors' table from the database
    c.execute("DROP TABLE sensors")
    print("Dropped 'sensors' table.")  # Confirmation message for table deletion
    
    # Verify that the 'sensors' table no longer exists
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sensors'")
    print("Table drop confirmed:", c.fetchall())  # Output should be empty if the table was dropped successfully
    
    # The cursor and connection are automatically closed after this block due to the use of 'with'


Created 'sensors' table.
Table creation confirmed: [('sensors',)]
All data in 'sensors' table:
   id         type  value         timestamp
0   1     Humidity  22.30  2024-11-30T04:59
1   2     Humidity  83.59  2024-11-16T15:24
2   3     Humidity  60.08  2024-11-03T11:58
3   4     Pressure  74.24  2024-11-06T05:58
4   5     Humidity  54.16  2024-11-21T01:19
5   6     Pressure  96.18  2024-11-03T11:29
6   7  Temperature  92.62  2024-11-19T13:48
7   8  Temperature  16.43  2024-11-05T12:12
8   9  Temperature  48.26  2024-11-14T22:49
9  10     Humidity  27.10  2024-11-24T04:07
Dropped 'sensors' table.
Table drop confirmed: []


In [2]:
# Create a list of tuples to form the DataFrame
# Sample data with year and sunactivity values
data = [
    (1700, 12.0),
    (1701, 15.3),
    (1702, 9.8),
    (1703, 21.1),
    (1704, 18.5),
    (1705, 22.7),  # Above 20, will be deleted later
    (1706, 19.9),
    (1707, 25.4),  # Above 20, will be deleted later
    (1708, 13.2),
    (1709, 16.8)
]

# Convert the list of tuples to a DataFrame for reference
df = pd.DataFrame(data, columns=["year", "sunactivity"])
print("Original DataFrame:")
print(df)

# Create an in-memory SQLite database and table
with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    
    # Create table without specifying data types
    c.execute("CREATE TABLE sunspots (year, sunactivity)")
    print("Created table 'sunspots' without specifying data types.")
    
    # Insert all records from the list of tuples into the table
    c.executemany("INSERT INTO sunspots (year, sunactivity) VALUES (?, ?)", data)
    
    # Confirm the row count after insertion
    c.execute("SELECT COUNT(*) FROM sunspots")
    row_count = c.fetchone()[0]
    print(f"Inserted {row_count} rows into 'sunspots' table.")
    
    # Delete records where sunactivity is greater than 20
    deleted_rows = c.execute("DELETE FROM sunspots WHERE sunactivity > 20").rowcount
    print(f"Deleted {deleted_rows} rows where sunactivity > 20.")
    
    # Query to select all data where year is less than 1732
    result_df = read_sql("SELECT * FROM sunspots WHERE year < 1732", con)
    print("Records where year is less than 1732:")
    print(result_df)
    
    # Cursor and connection are automatically closed at the end of the with block


Original DataFrame:
   year  sunactivity
0  1700         12.0
1  1701         15.3
2  1702          9.8
3  1703         21.1
4  1704         18.5
5  1705         22.7
6  1706         19.9
7  1707         25.4
8  1708         13.2
9  1709         16.8
Created table 'sunspots' without specifying data types.
Inserted 10 rows into 'sunspots' table.
Deleted 3 rows where sunactivity > 20.
Records where year is less than 1732:
   year  sunactivity
0  1700         12.0
1  1701         15.3
2  1702          9.8
3  1704         18.5
4  1706         19.9
5  1708         13.2
6  1709         16.8


In [3]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

# Create an in-memory SQLite database engine for demonstration purposes
engine = create_engine('sqlite:///:memory:')

# Define the base class for model definitions
Base = declarative_base()

# Define the Station table schema with SQLAlchemy ORM
class Station(Base):
    """
    Represents a Station entity in the database.

    Attributes:
        id (int): Unique identifier for each station, serves as the primary key.
        name (str): Name of the station, cannot be null.
        location (str): Physical location of the station, cannot be null.
        sensors (relationship): Establishes a one-to-many relationship with the Sensor class.
    """
    __tablename__ = 'stations'  # Name of the table in the database
    
    # Define columns in the table
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    name = Column(String, nullable=False)  # Station name
    location = Column(String, nullable=False)  # Station location
    
    # Define a one-to-many relationship with the Sensor table
    sensors = relationship("Sensor", back_populates="station")
    
    def __repr__(self):
        return f"Station(id={self.id}, name='{self.name}', location='{self.location}')"

# Define the Sensor table schema with SQLAlchemy ORM
class Sensor(Base):
    """
    Represents a Sensor entity in the database.

    Attributes:
        id (int): Unique identifier for each sensor, serves as the primary key.
        type (str): Type of sensor (e.g., Temperature, Humidity), cannot be null.
        value (float): Recorded value from the sensor, cannot be null.
        station_id (int): Foreign key linking this sensor to a specific station.
        station (relationship): Establishes a many-to-one relationship with the Station class.
    """
    __tablename__ = 'sensors'  # Name of the table in the database
    
    # Define columns in the table
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    type = Column(String, nullable=False)  # Sensor type (e.g., Temperature, Humidity)
    value = Column(Float, nullable=False)  # Sensor value
    station_id = Column(Integer, ForeignKey('stations.id'), nullable=False)  # Foreign key referencing stations table
    
    # Define a many-to-one relationship with the Station table
    station = relationship("Station", back_populates="sensors")
    
    def __repr__(self):
        return f"Sensor(id={self.id}, type='{self.type}', value={self.value}, station_id={self.station_id})"

# Create all tables in the database using metadata from the Base class
Base.metadata.create_all(engine)

# Set up a session to interact with the database
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Insert sample data into the stations and sensors tables
# Adding two stations: "Station A" at "North Pole" and "Station B" at "South Pole"
station1 = Station(name="Station A", location="North Pole")
station2 = Station(name="Station B", location="South Pole")
session.add_all([station1, station2])  # Add stations to the session
session.commit()  # Commit to save data to the database

# Insert sample data into the sensors table, linking the sensor to a station
sensor1 = Sensor(type="Temperature", value=15.5, station_id=station1.id)
session.add_all([sensor1])  # Add sensor to the session
session.commit()  # Commit to save data to the database

# Query and display all stations in the database
stations = session.query(Station).all()
print("Stations:")
for station in stations:
    print(f"ID: {station.id}, Name: {station.name}, Location: {station.location}")

# Query and display all sensors in the database
sensors = session.query(Sensor).all()
print("\nSensors:")
for sensor in sensors:
    print(f"ID: {sensor.id}, Type: {sensor.type}, Value: {sensor.value}, Station ID: {sensor.station_id}")

# Close the session to free up resources
session.close()


Stations:
ID: 1, Name: Station A, Location: North Pole
ID: 2, Name: Station B, Location: South Pole

Sensors:
ID: 1, Type: Temperature, Value: 15.5, Station ID: 1


In [4]:
# Insert another sensor into the sensors table, linking the sensor to a station
sensor2 = Sensor(type="Humidity", value=30.2, station_id=station2.id)
session.add_all([sensor2])  # Add sensor to the session

# Query the first row in the station table
first_station = session.query(Station).first()
print("First Station:", first_station)

# Query all rows in the station table
all_stations = session.query(Station).all()
print("\nAll Stations:")
for station in all_stations:
    print(f"ID: {station.id}, Name: {station.name}, Location: {station.location}")

# Query all rows in the sensor table
all_sensors = session.query(Sensor).all()
print("\nAll Sensors:")
for sensor in all_sensors:
    print(f"ID: {sensor.id}, Type: {sensor.type}, Value: {sensor.value}, Station ID: {sensor.station_id}")

# Query the first sensor associated with the first station
first_sensor = session.query(Sensor).filter(Sensor.station_id == first_station.id).first()
print("\nFirst Sensor for the First Station:")
print(f"ID: {first_sensor.id}, Type: {first_sensor.type}, Value: {first_sensor.value}, Station ID: {first_sensor.station_id}")

# Use pandas to query the station table and display it as a DataFrame
with engine.connect() as connection:
    stations_df = read_sql("SELECT * FROM stations", connection)
print("\nStations Table (using pandas read_sql):")
print(stations_df)

# Close the session after all database operations are complete
session.close()

First Station: Station(id=1, name='Station A', location='North Pole')

All Stations:
ID: 1, Name: Station A, Location: North Pole
ID: 2, Name: Station B, Location: South Pole

All Sensors:
ID: 1, Type: Temperature, Value: 15.5, Station ID: 1
ID: 2, Type: Humidity, Value: 30.2, Station ID: 2

First Sensor for the First Station:
ID: 1, Type: Temperature, Value: 15.5, Station ID: 1

Stations Table (using pandas read_sql):
   id       name    location
0   1  Station A  North Pole
1   2  Station B  South Pole
